In [1]:
# A* implementation for a grid maze (4-neighbours, unit costs).
import heapq
from typing import List, Tuple, Dict
import numpy as np

Coord = Tuple[int,int]

def neighbors_4(grid: np.ndarray, r: int, c: int) -> List[Coord]:
    H, W = grid.shape
    ans = []
    for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
        nr, nc = r+dr, c+dc
        if 0 <= nr < H and 0 <= nc < W and grid[nr, nc] != 1:
            ans.append((nr,nc))
    return ans

def reconstruct_path(came_from: Dict[Coord, Coord], current: Coord) -> List[Coord]:
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    path.reverse()
    return path

def astar(grid: np.ndarray, start: Coord, goal: Coord, h_func, move_cost=1):
    open_heap = []
    g = {start: 0.0}
    f = {start: h_func(start)}
    heapq.heappush(open_heap, (f[start], 0, start))
    came_from = {}
    closed = set()
    expansions = 0
    tie = 0
    while open_heap:
        _, _, current = heapq.heappop(open_heap)
        if current in closed:
            continue
        expansions += 1
        if current == goal:
            path = reconstruct_path(came_from, current)
            return path, g[current], expansions
        closed.add(current)
        r, c = current
        for nb in neighbors_4(grid, r, c):
            tentative_g = g[current] + move_cost
            if nb not in g or tentative_g < g[nb]:
                came_from[nb] = current
                g[nb] = tentative_g
                f[nb] = g[nb] + h_func(nb)
                tie += 1
                heapq.heappush(open_heap, (f[nb], tie, nb))
    return [], float('inf'), expansions

def dijkstra_grid(grid: np.ndarray, start: Coord, goal: Coord, move_cost=1):
    return astar(grid, start, goal, lambda _: 0.0, move_cost=move_cost)


In [2]:
maze = np.array([
 [0,0,0,1,0,0,0,0],
 [1,1,0,1,0,1,1,0],
 [0,0,0,0,0,0,1,0],
 [0,1,1,1,1,0,1,0],
 [0,0,0,0,1,0,0,0],
 [0,1,1,0,0,0,1,0],
 [0,0,0,0,1,0,0,0],
 [0,1,0,0,0,0,1,0],
], dtype=int)
start = (0,0)
goal  = (7,7)


In [3]:
def h_manhattan(s): return abs(s[0]-goal[0]) + abs(s[1]-goal[1])


In [4]:
def h_scaled_1p5(s): return 1.5 * (abs(s[0]-goal[0]) + abs(s[1]-goal[1]))


In [5]:
bad_node = (2,4)
def h_locally_inconsistent(s):
    base = abs(s[0]-goal[0]) + abs(s[1]-goal[1])
    return float(base + 3) if s == bad_node else float(base)
